<a href="https://colab.research.google.com/github/Shujaat123/ACP-LSTM-NFR/blob/main/ACP_LSTM_NFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Anticancer Peptides Classification Using Long-Short-Term Memory with Novel Feature Representation

Import Libraries and download pre-trained model

In [1]:
import numpy as np
import tensorflow
## Perfmetrics

!pip install wget
import wget

import os

# Define the file paths
feature_extraction_inference_path = 'feature_extraction_inference.py'
best_model_path = 'model_acp740_4.keras'

# Check if 'feature_extraction_inference.py' already exists and delete it
if os.path.exists(feature_extraction_inference_path):
    os.remove(feature_extraction_inference_path)
    print(f"Existing '{feature_extraction_inference_path}' has been deleted.")

# Download the file again
feature_extraction_inference = wget.download('https://raw.githubusercontent.com/Shujaat123/ACP-LSTM-NFR/main/feature_extraction_inference.py')
print(f"Downloaded: {feature_extraction_inference}")

# Check if 'model_acp740_4.keras' already exists and delete it
if os.path.exists(best_model_path):
    os.remove(best_model_path)
    print(f"Existing '{best_model_path}' has been deleted.")

# Download the model file again
best_model = wget.download('https://raw.githubusercontent.com/Shujaat123/ACP-LSTM-NFR/main/best_model/model_acp740_4.keras')
print(f"Downloaded: {best_model}")

from keras.models import load_model
# Load the best saved model from the specified file path
model = load_model('model_acp740_4.keras')

############################################################################
from feature_extraction_inference import load_sequences, proposed_features, ACP_DL_feature

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=50134a1fdcbd0320da7c6eed68a65df6147b1c46c39d247e54364717df653fef
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Downloaded: feature_extraction_inference.py
Downloaded: model_acp740_4.keras


Upload peptide sequence in FASTA format.
Program only accept .txt files
A sample file should contain


--------------
\>[Name-of-Peptide]
>[sequence]


e.g.,

\>Seq1
>GLWSKIKEVGKEAV



In [2]:
import os
from google.colab import files

# Function to handle file upload and restrictions
def upload_txt_file():
    # Check if 'sample_seq.txt' already exists and delete it if present
    if os.path.exists('sample_seq.txt'):
        os.remove('sample_seq.txt')
        print("Existing 'sample_seq.txt' has been deleted.")

    # Upload the file
    uploaded = files.upload()  # This will prompt the user to upload a file

    # Check if exactly one file is uploaded and if it's a .txt file
    if len(uploaded) != 1:
        raise Exception("Please upload exactly one file.")

    # Get the name of the uploaded file
    filename = list(uploaded.keys())[0]

    # Check if the uploaded file has a .txt extension
    if not filename.endswith('.txt'):
        raise Exception("Please upload a .txt file only.")

    # Save the uploaded file as 'sample_seq.txt'
    with open('sample_seq.txt', 'wb') as f:
        f.write(uploaded[filename])

    print(f"File '{filename}' has been successfully saved as 'sample_seq.txt'.")

# Call the function to upload and save the file
upload_txt_file()


Saving sample_seq.txt to sample_seq.txt
File 'sample_seq.txt' has been successfully saved as 'sample_seq.txt'.


Use uploaded sequence file and perform predictions using pre-trained model

In [3]:
path = 'sample_seq.txt'
gap1=1
gap2=5
folds = 5


# define parameters
timesteps = 1
batch_size = 64
epochs = 30

# get data
DataX, Label = proposed_features(load_sequences(path), gap1=gap1,gap2=gap2) # proposed novel features
bpf, kmer = ACP_DL_feature(path)  # features from ACP-DL method
X = np.concatenate((bpf, kmer), axis=1) # k-mer with binary
X = np.concatenate((X, DataX), axis=1)  # add side chain
data_dim = len(X[0])
#  expected input data shape: (batch_size, timesteps, data_dim)
X = np.reshape(X, (len(X), timesteps, data_dim))


lstm_proba = model.predict(X)

# print(lstm_proba)

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step


Display Prediction Results

In [4]:
lstm_proba = np.asarray(lstm_proba).squeeze()

import pandas as pd

# Create a pandas DataFrame combining both lists
df = pd.DataFrame({
    'Label': Label,
    'LSTM_Proba': lstm_proba
})

# Display the DataFrame
df

,Label,LSTM_Proba
0,NAME-of-Peptide\n,0.789894
1,ACP_1|1\n,0.898592
2,ACP_2|1\n,0.959864
3,ACP_3|1\n,0.944061
4,ACP_4|1\n,0.875278
...,...,...
736,non-ACP_360|0\n,0.023803
737,non-ACP_361|0\n,0.004193
738,non-ACP_362|0\n,0.007296
739,non-ACP_363|0\n,0.009932


Download prediction probabilities in Excel Sheet format

In [5]:
# Save the DataFrame as an Excel file
df.to_excel('results.xlsx', index=False)

# Download the Excel file
files.download('results.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>